In [1]:
from random import normalvariate

from faker import Faker
from texttable import Texttable

faker = Faker()

In [2]:
def print_table(headers, rows):
    table = Texttable()
    table.add_rows([headers] + rows)
    print(table.draw())

In [3]:
class Employee():
    def __init__(self):
        self.name = faker.name()
        self.email = faker.email()
        self.salary = int(max(1000, normalvariate(2000, 2000)))

In [4]:
with open('./employees.csv', 'w') as f:
    f.write('name,email,salary\n')
    for _ in range(1000):
        e = Employee()
        f.write(f'{e.name},{e.email},{e.salary}\n')

In [5]:
!head -n 10 ./employees.csv

name,email,salary
Elizabeth Payne,ftorres@yahoo.com,1000
Sarah Fox,robinadams@combs.biz,2089
Anthony Gates,brianknight@gmail.com,1000
Kristina Sparks,colleen56@edwards.com,2405
Laura Gordon,amandabenjamin@yahoo.com,1057
Jeffrey Barrett,taylorryan@williams.com,2062
Russell Taylor,todd49@hotmail.com,1000
Heather Ryan,fmora@dudley.net,3649
John Sanchez,robert89@ray.biz,1631


## Select all with simple filter

```sql
SELECT 
    * 
FROM 
    employees 
WHERE 
    salary > 2000 
```

In [6]:
with open('./employees.csv', 'r') as f:
    headers = [h for h in f.readline().split(',')]
    
    rows = []
    for raw in f.readlines():
        name, email, salary = raw.split(',')
        
        if int(salary) > 2000:
            rows.append([name, email, salary])
        
print_table(headers, rows)

+--------------------------+-------------------------------------+--------+
|           name           |                email                | salary |
|                          |                                     |        |
+==========================+=====================================+========+
| Sarah Fox                | robinadams@combs.biz                | 2089   |
+--------------------------+-------------------------------------+--------+
| Kristina Sparks          | colleen56@edwards.com               | 2405   |
+--------------------------+-------------------------------------+--------+
| Jeffrey Barrett          | taylorryan@williams.com             | 2062   |
+--------------------------+-------------------------------------+--------+
| Heather Ryan             | fmora@dudley.net                    | 3649   |
+--------------------------+-------------------------------------+--------+
| Amber Garcia             | dwatson@gmail.com                   | 2751   |
+-----------

## Select all with universal filter

```sql
SELECT 
    * 
FROM 
    employees 
WHERE 
    salary > 2000 and salary < 4000
```

In [7]:
def where(condition):
    with open('./employees.csv', 'r') as f:
        headers = f.readline().split(',')

        rows = []
        for raw in f.readlines():
            name, email, salary = raw.split(',')
            
            if condition(dict(name=name, email=email, salary=int(salary))):
                rows.append([name, email, salary])
        
    print_table(headers, rows)
    
where(lambda x: x['salary'] > 2000 and x['salary'] < 4000)    

+--------------------------+-------------------------------------+--------+
|           name           |                email                | salary |
|                          |                                     |        |
+==========================+=====================================+========+
| Sarah Fox                | robinadams@combs.biz                | 2089   |
+--------------------------+-------------------------------------+--------+
| Kristina Sparks          | colleen56@edwards.com               | 2405   |
+--------------------------+-------------------------------------+--------+
| Jeffrey Barrett          | taylorryan@williams.com             | 2062   |
+--------------------------+-------------------------------------+--------+
| Heather Ryan             | fmora@dudley.net                    | 3649   |
+--------------------------+-------------------------------------+--------+
| Amber Garcia             | dwatson@gmail.com                   | 2751   |
+-----------

## Complex query

```sql
SELECT 
    name,
    salary
FROM 
    employees 
WHERE 
    name LIKE B%
ORDER BY salary
LIMIT 5
```

In [53]:
(Table('employees.csv')    
    .select('name', 'salary')
    .where(lambda x: x['name'].startswith('B'))
    .order_by('salary', asc=False)
    .limit(5)
    .collect())

+--------------------+--------+
|        name        | salary |
+====================+========+
| Brittnee Miller    | 5483   |
+--------------------+--------+
| Burt Wiza          | 5332   |
+--------------------+--------+
| Blanca Runolfsson  | 4682   |
+--------------------+--------+
| Berniece Muller MD | 4128   |
+--------------------+--------+
| Bolden Keeling     | 3856   |
+--------------------+--------+


In [52]:
class Table:
    
    def __init__(self, filepath):
        with open(filepath, 'r') as f:
            self.columns = f.readline().strip().split(',')
            self.rows = []
            
            for line in f.readlines():
                entries = line.strip().split(',')
                
                self.rows.append({
                    column: entries[i]
                    for i, column in enumerate(self.columns)
                })
            
    def select(self, *columns):
        self.columns = columns
        
        return self
    
    def limit(self, limit):
        self.rows = self.rows[:limit]

        return self
    
    def offset(self, offset):
        self.rows = self.rows[offset:]
    
        return self
    
    def order_by(self, column, asc=True):
        self.rows = sorted(self.rows, key=lambda x: x[column], reverse=not asc)
        
        return self
    
    def where(self, condition):
        self.rows = [row for row in self.rows if condition(row)]
        
        return self
    
    def collect(self):        
        print_table(
            self.columns, 
            [[row[column] for column in self.columns] for row in self.rows])